In [2]:
import pandas as pd
import quandl
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
from jupyterthemes import jtplot
import math
quandl.ApiConfig.api_key = 'SVCwUsxMpuF3eQ-URgUT'

C:\Users\runmd\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
start = datetime.datetime(2010, 1, 1)

# end = datetime.datetime(2013, 1, 27)

df = web.DataReader("GOOG", 'yahoo', start)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-12-31,310.356445,310.679321,307.986847,307.986847,307.986847,2455400
2010-01-04,311.449310,312.721039,310.103088,311.349976,311.349976,3937800
2010-01-05,311.563568,311.891449,308.761810,309.978882,309.978882,6048500
2010-01-06,310.907837,310.907837,301.220856,302.164703,302.164703,8009000
2010-01-07,302.731018,303.029083,294.410156,295.130463,295.130463,12912000


In [10]:
df.to_csv('googledata.csv')
df.tail()

,Open,High,Low,Close,Adj Close,Volume,HL_PCT,Return,Label
Date,,,,,,,,,
2017-12-08,1037.489990,1042.050049,1032.521973,1037.050049,1037.050049,1290800,0.922796,0.005936,NaN
2017-12-11,1035.500000,1043.800049,1032.050049,1041.099976,1041.099976,1192800,1.138511,0.003905,NaN
2017-12-12,1039.630005,1050.310059,1033.689941,1040.479980,1040.479980,1279500,1.607844,-0.000596,NaN
2017-12-13,1046.119995,1046.665039,1038.380005,1040.609985,1040.609985,1219800,0.797881,0.000125,NaN
2017-12-15,1054.609985,1067.619995,1050.189941,1063.750000,1063.750000,2139996,1.659705,0.022237,NaN


In [5]:
df['HL_PCT'] = (df['High'] - df['Low']) / df['Low'] * 100.0
df['Return'] = df['Adj Close'].pct_change()

In [6]:
df.fillna(-99999,inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume,HL_PCT,Return
Date,,,,,,,,
2009-12-31,310.356445,310.679321,307.986847,307.986847,307.986847,2455400,0.874217,-99999.000000
2010-01-04,311.449310,312.721039,310.103088,311.349976,311.349976,3937800,0.844220,0.010920
2010-01-05,311.563568,311.891449,308.761810,309.978882,309.978882,6048500,1.013609,-0.004404
2010-01-06,310.907837,310.907837,301.220856,302.164703,302.164703,8009000,3.215906,-0.025209
2010-01-07,302.731018,303.029083,294.410156,295.130463,295.130463,12912000,2.927524,-0.023279


In [7]:

forecast_out = 10
df['Label'] = df['Adj Close'].shift(-forecast_out)


In [8]:
df[:][-20:]

,Open,High,Low,Close,Adj Close,Volume,HL_PCT,Return,Label
Date,,,,,,,,,
2017-11-16,1022.520020,1035.920044,1022.520020,1032.500000,1032.500000,1129700,1.310490,0.011353,1010.169983
2017-11-17,1034.010010,1034.420044,1017.750000,1019.090027,1019.090027,1397100,1.637931,-0.012988,998.679993
2017-11-20,1020.260010,1022.609985,1017.500000,1018.380005,1018.380005,953500,0.502210,-0.000697,1005.150024
2017-11-21,1023.309998,1035.109985,1022.655029,1034.489990,1034.489990,1097000,1.217904,0.015819,1018.380005
2017-11-22,1035.000000,1039.706055,1031.430054,1035.959961,1035.959961,746300,0.802381,0.001421,1030.930054
2017-11-24,1035.869995,1043.177979,1035.000000,1040.609985,1040.609985,537000,0.790143,0.004489,1037.050049
2017-11-27,1040.000000,1055.459961,1038.439941,1054.209961,1054.209961,1307900,1.638999,0.013069,1041.099976
2017-11-28,1055.089966,1062.375000,1040.000000,1047.410034,1047.410034,1424400,2.151442,-0.006450,1040.479980
2017-11-29,1042.680054,1044.079956,1015.650024,1021.659973,1021.659973,2459400,2.799186,-0.024585,1040.609985


In [9]:
X = np.array(df[['Adj Close','HL_PCT','Return']])
y = np.array(df['Label'])
X = preprocessing.scale(X)
X = X[:-forecast_out]


In [ ]:
X_train, X_text, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)